In [7]:
import pymongo
import json
import jieba
import re
import pymysql

In [8]:
def mongo_connect_resume(db_name, col_name):
    
    global col
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") # 跟mongodb建立連

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    col = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立
    
def resume_find():
    # 尋找多筆資料
    return col.find({},{"_id":0})

In [16]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") # 跟mongodb建立連

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立
    
def data_find():
    # 尋找多筆資料
    return mycol.find({}, {"_id":0})

In [9]:
def resume_jieba_list(resume):
    resume_list = []
    for i in resume:
        for j in i['value']:
            resume_list.append(j)
    return resume_list

In [17]:
def create_w2v_list(job_data):
    w2v_list = []
    for i in job_data:
        content_list = []
        content_list.append(i['jobURL'])
        for j in i['jobDescription_concate_jieba']:
            for k in j:
                content_list.append(k)
            w2v_list.append(content_list)
    return w2v_list

In [18]:
mongo_connect_build('jieba_clean', 'job_clean') 
job_data = data_find()
w2v_list = create_w2v_list(job_data)

In [10]:
mongo_connect_resume('cakeresume','jieba_resume')
resume_data = resume_find()
resume_list = resume_jieba_list(resume_data)

In [12]:
import gensim
from gensim.models import Word2Vec

In [13]:
model = gensim.models.Word2Vec.load('./w2v_test_1026.model')

In [14]:
def create_resume_for_similar(resume_jieba):
    resume_list_all = []
    for word_list in resume_jieba:
        worker_list= []
        for word in word_list:
            if word in model.wv.vocab.keys():
                worker_list.append(word)
            else:
                pass
        resume_list_all.append(worker_list)
    return resume_list_all

In [15]:
resume_list_all = create_resume_for_similar(resume_list)

In [25]:
resume_list_all[0]

['Wang',
 '產品',
 '總監',
 '遊戲',
 '機率',
 '工程',
 'Taipei',
 'Special',
 'TW',
 'protected',
 '十多',
 '遊戲',
 '開發',
 '擔任',
 '遊戲',
 '研發',
 '主管',
 '企劃',
 '主管',
 '敏捷',
 '專案',
 '機率',
 '工程',
 '職務',
 '擅長',
 '零到',
 '撰寫',
 '企劃',
 '文案',
 '專案',
 '開發',
 '產品',
 '經歷',
 '建置',
 '專案',
 '管理',
 '溝通',
 '協調',
 '敏捷',
 '開發',
 '進度',
 '控管',
 '產品',
 '調研',
 '整合',
 '可行性',
 '評估',
 '原型',
 '遊戲',
 '文件',
 '流程',
 '規劃',
 '數值',
 '機率',
 '音樂',
 '音效',
 '測試',
 '驗證',
 '產品',
 '上線',
 '數據',
 '分析',
 '優化',
 '開發',
 '遊戲',
 '棋牌',
 '牛牛',
 '麻將',
 '龍虎',
 '捕魚機',
 '老虎',
 '籃球',
 '棒球',
 '保齡球',
 '攀岩',
 '賽車',
 '車等',
 '遊戲',
 '電腦',
 '網頁',
 '手機',
 '平台',
 '專案',
 '數數',
 '十款',
 '數值',
 '機率',
 'AN',
 '捕魚',
 '遊戲',
 '專案',
 '機率',
 '計算',
 '十款',
 '老虎',
 '專案',
 '研發',
 '遊戲',
 '設計',
 '數學',
 '模型',
 '建立',
 'EXCEL',
 'VBA',
 '撰寫',
 '程式',
 '巨集',
 '模擬',
 '老虎',
 '遊戲',
 '列印',
 '數據',
 '產出',
 '報表',
 '繪製',
 '波動',
 '曲線',
 '用戶',
 '用戶',
 'UI',
 'UX',
 '用戶',
 '觀察',
 '最小',
 '觀察',
 '遊戲',
 '設計',
 '框圖',
 'Wireframe',
 '原型',
 'Mock',
 '作業',
 '完善',
 '字型',
 '顏色',
 '原型',
 '工具',
 'Axure

In [20]:
def job_mapping(w2v_list,user_list):
    rate = {}
    for i in w2v_list:
        a = model.n_similarity(i,user_list)
        k = i[0]
        v = a
        rate[k] = v
    rate_sort = sorted(rate.items(), key=lambda x:x[1],reverse=True)
    url = []
    for i in range(0,10):
        url.append(rate_sort[i][0])

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
print(sorted(rate.items(), key=lambda x:x[1],reverse=True))

[('72r0a', 0.9772649), ('719s2', 0.9742383), ('24jbo', 0.9729541), ('6qeyr', 0.9728642), ('579x6', 0.97096896), ('6swxd', 0.9700934), ('6usvf', 0.968573), ('4u3e1', 0.9671184), ('6qaoe', 0.967057), ('72lo3', 0.96658486), ('6hi86', 0.9662853), ('6ut56', 0.9662644), ('70da5', 0.9662329), ('70epy', 0.96563834), ('6kadr', 0.9656303), ('6y83l', 0.96559817), ('6q2o4', 0.9645815), ('6wdhr', 0.96320367), ('6cg8q', 0.9620268), ('726i5', 0.9619917), ('6xox5', 0.96134335), ('6ji5k', 0.9609855), ('70j8t', 0.95967937), ('6vlf8', 0.9595451), ('70xeq', 0.95910907), ('4qghx', 0.9582243), ('6wt7i', 0.95727736), ('4skrb', 0.95689833), ('6knu1', 0.9566814), ('6xlo7', 0.95454407), ('5vyz2', 0.9539386), ('6zteg', 0.9537606), ('6wquv', 0.95344216), ('5pvxj', 0.9528709), ('725wh', 0.9525481), ('6m5hx', 0.9518035), ('6rjy7', 0.9516543), ('6mtrx', 0.95084065), ('707aj', 0.9505564), ('6mpow', 0.950383), ('70cv8', 0.9503639), ('6q5r7', 0.9493597), ('6u48l', 0.9493135), ('6tnrh', 0.9488227), ('5eckh', 0.9487554),

In [30]:
rate_sort = sorted(rate.items(), key=lambda x:x[1],reverse=True)

In [36]:
url = []
for i in range(0,10):
    url.append(rate_sort[i][0])

In [38]:
def connect_to_mysql(user,passwd,db):
    # 設定資料庫連線資訊
    db_setting = {
        "host" : 'localhost',
        "port" : 3306,
        "user" : user,
        "passwd" : passwd,
        "db" : db,
        "charset" : 'utf8mb4'
        }
    conn = pymysql.connect(**db_setting)
#     print("Mysql connect")
    return conn

In [39]:
def mysql_select_data(Job_ID):
    

    for i in Job_ID:
        try: 
            conn = connect_to_mysql('root','frank0813','topic_104')  # 連接資料庫
            cursor = conn.cursor()
            sql = """
            SELECT Job_ID,Job_Name,jobCat_main,Company,Work_area_clean,workExp_clean,salary_clean FROM job_104 
            where Job_ID = '%s'; 
            """ % (i)

            cursor.execute(sql)
            data = cursor.fetchall()
            print(data)
            print("")
            
            print("======================================")
            cursor.close()
            conn.close()
        except:
            pass
        

In [40]:
mysql_select_data(url)

(('72r0a', '遊戲產品企劃', '行銷╱企劃╱專案管理類', 'ible Technology Inc_鴻辰通訊股份有限公司', '台北市', '3', '待遇面議,0,0'),)

(('719s2', '遊戲企劃', '行銷╱企劃╱專案管理類', '未來軟體開發有限公司', '台北市', '2', '待遇面議,0,0'),)

(('24jbo', '遊戲企劃專員', '行銷╱企劃╱專案管理類', '鈊象電子股份有限公司', '新北市', '0', '月薪,24000,40000'),)

(('6qeyr', '遊戲研發企劃(企劃助理)', '行銷╱企劃╱專案管理類', '傳奇網路遊戲股份有限公司', '台北市', '0', '月薪,28000,30000'),)

(('579x6', '遊戲研發企劃', '行銷╱企劃╱專案管理類', '傳奇網路遊戲股份有限公司', '台北市', '0', '月薪,30000,36000'),)

(('6swxd', 'YY-遊戲策劃', '行銷╱企劃╱專案管理類', '傲世娛樂文創有限公司', '台北市', '3', '月薪,60000,100000'),)

(('6usvf', '遊戲研發企劃', '行銷╱企劃╱專案管理類', '火鳳天翔科技有限公司', '新北市', '2', '月薪,40000,60000'),)

(('4u3e1', '手機或網頁遊戲營運PM', '行銷╱企劃╱專案管理類', '異軍互動娛樂股份有限公司', '台北市', '2', '月薪,35000,50000'),)

(('6qaoe', '遊戲企劃', '行銷╱企劃╱專案管理類', '馬來西亞商韋博科技有限公司台灣分公司', '新北市', '2', '待遇面議,0,0'),)

(('72lo3', '【B2B合法牌照投資集團-游戲策劃/台北/正職】', '行銷╱企劃╱專案管理類', '康彼斯顧問股份有限公司', '台北市', '3', '待遇面議,0,0'),)

